## Datasets to be evaluated Visualization

In [1]:
import yaml 

with open('../config/scenarios.yaml') as file: 
    scenarios = yaml.safe_load(file)

In [2]:
scenarios['synth']

{'n_per': 1000,
 'n_blobs': [10, 5, 5],
 'n_episodes': 3,
 'latent_dim': 2,
 'drift_bin': 0.0}

In [3]:
import cf_cl.datasets as data_utils

In [4]:
import importlib 
importlib.reload(data_utils)

<module 'cf_cl.datasets' from '/home/euecrlf_wsl/repos/cf-cl/src/cf_cl/datasets.py'>

In [5]:
benchmark = data_utils.create_synthetic_benchmark(
    **scenarios['synth']
)

In [6]:
train_stream = benchmark.train_stream

In [7]:
for experience in benchmark.train_stream:
    print(f"Start of experience {experience.current_experience}")

Start of experience 0
Start of experience 1
Start of experience 2


## Classic Benchmark 

In [8]:
dataset_named = data_utils.NamedDataset("ccifar10", "SplitCIFAR10")

In [9]:
classic_benchmarks = data_utils.create_classic_avalanche_benchmark(
    [dataset_named], 
    n_experiences=5
)

In [10]:
classic_benchmarks

{'ccifar10': <avalanche.benchmarks.scenarios.deprecated.new_classes.nc_scenario.NCScenario at 0x73676400b6d0>}

In [11]:
classic_benchmarks

{'ccifar10': <avalanche.benchmarks.scenarios.deprecated.new_classes.nc_scenario.NCScenario at 0x73676400b6d0>}

In [12]:
for experience in classic_benchmarks[dataset_named.file].train_stream:
    print(f"Start of experience {experience.current_experience}")


Start of experience 0
Start of experience 1
Start of experience 2
Start of experience 3
Start of experience 4


## Train

In [13]:
from cf_cl.vae import VAE, MlpVAE, VAE_loss

In [34]:
from torch.optim import SGD 

from avalanche.training.supervised import (Naive, CWRStar, Replay, GDumb, 
                                           Cumulative, LwF, GEM, AGEM, EWC, 
                                           VAETraining)

model = VAE(img_channels=3, img_size=32, latent_dim=2)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = VAE_loss

In [35]:
from avalanche.evaluation.metrics import (
    accuracy_metrics, loss_metrics, forgetting_metrics, bwt_metrics, class_accuracy_metrics, timing_metrics
)
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins.evaluation import EvaluationPlugin

interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    *loss_metrics(minibatch=False, epoch=False, experience=True, stream=True),
    *timing_metrics(experience=True, stream=True),  # optional
    loggers=[interactive_logger],
)


In [39]:
cl_strategy = VAETraining( #SupervisedTemplate overwritten criterion for specific input
    model=model, optimizer=optimizer, criterion=criterion,
    train_mb_size=1, train_epochs=4, eval_mb_size=None,
    evaluator=eval_plugin
)

In [40]:
benchmark = classic_benchmarks[dataset_named.file]

In [41]:
print('Starting experiment...')
all_results = []

for exp in benchmark.train_stream:
    print(f"Start of experience: {exp.current_experience}")
    print(f"Current classes: {exp.classes_in_this_experience}")

    # Train on this experience
    cl_strategy.train(exp)
    print('Training completed')

    # Evaluate on the whole test stream
    print('Computing accuracy on the whole test set...')
    eval_results = cl_strategy.eval(benchmark.test_stream)
    all_results.append(eval_results)
    print({k: eval_results[k] for k in eval_results if 'Loss' in k})


Starting experiment...
Start of experience: 0
Current classes: [9, 5]
-- >> Start of training phase << --
100%|██████████| 10000/10000 [02:32<00:00, 65.42it/s]
Epoch 0 ended.
100%|██████████| 10000/10000 [02:39<00:00, 62.51it/s]
Epoch 1 ended.
100%|██████████| 10000/10000 [02:37<00:00, 63.66it/s]
Epoch 2 ended.
100%|██████████| 10000/10000 [02:59<00:00, 55.87it/s]
Epoch 3 ended.
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set...
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 2000/2000 [00:15<00:00, 129.96it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 5203.8508
	Time_Exp/eval_phase/test_stream/Task000/Exp000 = 15.3917
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 2000/2000 [00:16<00:00, 123.23it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_str

In [ ]:
all_results